In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Pipelines: Google Cloud Pipeline Components

#### Custom Training Image를 활용해서 모델을 학습하고, 등록, 배포하는 파이프라인을 구성
- custom training image 생성과 학습은 clip04에서 복습 가능

## 설치

필요한 library 설치

In [ ]:
!pip3 install --upgrade 'protobuf<4' \
                                google-cloud-aiplatform \
                                 google-cloud-storage \
                                 'kfp<2' \
                                 'google-cloud-pipeline-components<2'

## 환경 설정

Project ID와 Region 설정 

In [ ]:
!gcloud config list

In [ ]:
PROJECT_ID = "inspired-micron-414202" 
REGION = "us-central1"

# Project ID 세팅
! gcloud config set project {PROJECT_ID}


### - Cloud Storage bucket 생성

dataset이나 artifact를 저장하기 위한 bucket을 생성

In [ ]:
BUCKET_URI = f"gs://fs-practice-{PROJECT_ID}"  # @param {type:"string"}

In [ ]:
! gsutil mb -l {REGION} -p {PROJECT_ID} {BUCKET_URI}

### - Service Account 설정
Service Account가 Bucket에 접근할 수 있도록 설정

In [ ]:
shell_output = !gcloud auth list 2>/dev/null
SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator $BUCKET_URI
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer $BUCKET_URI

## 주요 Library Import (kuberflow pipeline) 


In [ ]:
from typing import Any, Dict, List

import google.cloud.aiplatform as aip
import kfp
from kfp.v2 import compiler

import random
import string

Vertext AI Pipeline root 설정

In [ ]:
PIPELINE_ROOT = "{}/pipeline_root/intro".format(BUCKET_URI)

Initialize AI platform object

In [ ]:
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

## Google Cloud Pipeline Components을 사용하여 Custom Pipeline을 정의

experimental.run_as_aiplatform_custom_job 메서드는 이전에 정의한 구성 요소와 worker_pool_specs 목록(이 경우 하나)을 인수로 받아 사용자 지정 훈련 작업이 구성된다.

그런 다음 google_cloud_pipeline_components 구성 요소를 사용하여 나머지 파이프라인을 정의합니다: 모델 업로드, 엔드포인트 생성 및 모델을 엔드포인트에 배포합니다.



#### Parameter 설정

In [ ]:
hp_dict: str = '{"num_hidden_layers": 1, "hidden_size": 16, "learning_rate": 0.01, "epochs": 1, "steps_per_epoch": -1}'
data_dir: str = "gs://aju-dev-demos-codelabs/bikes_weather/"
TRAINER_ARGS = ["--data-dir", data_dir, "--hptune-dict", hp_dict]


#### working directory 변수와 model endpoint 변수명을 생성
UUID를 랜덤하게 생성해서 변수들을 매번 다르게 설정

In [ ]:
UUID = "".join(random.choices(string.ascii_lowercase + string.digits, k=8))
WORKING_DIR = f"{PIPELINE_ROOT}/{UUID}"
MODEL_DISPLAY_NAME = f"fc_first_train_deploy{UUID}"
print(TRAINER_ARGS, WORKING_DIR, MODEL_DISPLAY_NAME)


#### Pipeline을 정의

![pipeline 구성](practice_image/Clip6_image.png)

In [ ]:
@kfp.dsl.pipeline(name="fc-first-train-endpoint-deploy" + UUID)
def pipeline(
    project: str = PROJECT_ID,
    model_display_name: str = MODEL_DISPLAY_NAME,
    serving_container_image_uri: str = "us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-9:latest",
):
    from google_cloud_pipeline_components.types import artifact_types
    from google_cloud_pipeline_components.v1.custom_job import \
        CustomTrainingJobOp
    from google_cloud_pipeline_components.v1.endpoint import (EndpointCreateOp,
                                                              ModelDeployOp)
    from google_cloud_pipeline_components.v1.model import ModelUploadOp
    from kfp.v2.components import importer_node

    custom_job_task = CustomTrainingJobOp(
        project=project,
        display_name="model-training",
        worker_pool_specs=[
            {
                "containerSpec": {
                    "args": TRAINER_ARGS,
                    "env": [{"name": "AIP_MODEL_DIR", "value": WORKING_DIR}],
                    "imageUri": "gcr.io/google-samples/bw-cc-train:latest",
                },
                "replicaCount": "1",
                "machineSpec": {
                    "machineType": "n1-standard-16"
                },
            }
        ],
    )

    import_unmanaged_model_task = importer_node.importer(
        artifact_uri=WORKING_DIR,
        artifact_class=artifact_types.UnmanagedContainerModel,
        metadata={
            "containerSpec": {
                "imageUri": "us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-9:latest",
            },
        },
    ).after(custom_job_task)

    model_upload_op = ModelUploadOp(
        project=project,
        display_name=model_display_name,
        unmanaged_container_model=import_unmanaged_model_task.outputs["artifact"],
    )
    model_upload_op.after(import_unmanaged_model_task)

    endpoint_create_op = EndpointCreateOp(
        project=project,
        display_name="first-fc-pipelines-created-endpoint",
    )

    ModelDeployOp(
        endpoint=endpoint_create_op.outputs["endpoint"],
        model=model_upload_op.outputs["model"],
        deployed_model_display_name=model_display_name,
        dedicated_resources_machine_type="n1-standard-16",
        dedicated_resources_min_replica_count=1,
        dedicated_resources_max_replica_count=1,
    )

## Pipeline 실행

In [ ]:
compiler.Compiler().compile(
    pipeline_func=pipeline,
    package_path="fc_first_model_training_pipeline.json",
)

In [ ]:
DISPLAY_NAME = "first_fc_pipelines_training_" + UUID

job = aip.PipelineJob(
    display_name=DISPLAY_NAME,
    template_path="fc_first_model_training_pipeline.json",
    pipeline_root=PIPELINE_ROOT,
    enable_caching=False,
)

job.run(service_account=SERVICE_ACCOUNT)

! rm fc_model_training_pipeline.json

## SDK를 활용해서 Pipeline 정보 확인 및 컨트롤

In [ ]:
def get_task_detail(
    task_details: List[Dict[str, Any]], task_name: str
) -> List[Dict[str, Any]]:
    for task_detail in task_details:
        if task_detail.task_name == task_name:
            return task_detail

#### pipeline detail 확인

In [ ]:
pipeline_task_details = (
    job.gca_resource.job_detail.task_details
)
pipeline_task_details

#### endpoint 확인 

In [ ]:
endpoint_task = get_task_detail(pipeline_task_details, "endpoint-create")
endpoint_resourceName = (
    endpoint_task.outputs["endpoint"].artifacts[0].metadata["resourceName"]
)
endpoint = aip.Endpoint(endpoint_resourceName)
endpoint

#### endpoint undeploy와 삭제

In [ ]:
endpoint.undeploy_all()
endpoint.delete()

#### pipeline의 모델과 모델 제거하기

In [ ]:
model_task = get_task_detail(pipeline_task_details, "model-upload")
model_resourceName = model_task.outputs["model"].artifacts[0].metadata["resourceName"]
model = aip.Model(model_resourceName)
model.delete()

#### Pipline 제거하기

In [ ]:
job.delete()